# Training Notebook

In [96]:
import glob
import os
import numpy as np

# Vision
import cv2

## Utility
import xml.etree.ElementTree as ET
from tqdm import tqdm

import inference_utils

## Displaying in-cell
from IPython.display import Image
import matplotlib.pyplot as plt

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [2]:
class FishCounter(object):
    import cv2

    def __init__(self, model_config, model_weights):
        self.net = cv2.dnn.readNetFromDarknet(model_config, model_weights)
        self.input_width = 416
        self.input_height = 416

    def inference(self, frame):
        """Run inference on single frame."""
        # Create a 4D blob from a frame.
        blob = cv2.dnn.blobFromImage(
            frame,
            1 / 255,
            (self.input_width, self.input_height),
            [0, 0, 0],
            1,
            crop=False,
        )

        # Sets the input to the network
        self.net.setInput(blob)

        # Runs the forward pass to get output of the output layers
        outs = self.net.forward(self._get_outputs_names(net=self.net))

    def _get_outputs_names(net):
        """Get the names of the output layers."""
        # Get the names of all the layers in the network
        layersNames = net.getLayerNames()
        # Get the names of the output layers, i.e. the layers with unconnected outputs
        return [layersNames[i - 1] for i in net.getUnconnectedOutLayers()]

    def get_annotated_frame(self):
        """Returns main frame with fish highlighted."""
        frame = self.frame.copy()

In [111]:
def get_dimensions_of_annotations(fpath):
    """Extracts dimensions of annotation boxes from xml file for image."""

    class_label = "herring"  # TODO: Will need to accomodate multiple classes

    tree = ET.parse(fpath)
    root = tree.getroot()

    xmins = []
    ymins = []
    xmaxs = []
    ymaxs = []
    for o in root.iter("object"):
        if o.find("name").text == class_label:
            xmins.append(eval(o[4][0].text))
            ymins.append(eval(o[4][1].text))
            xmaxs.append(eval(o[4][2].text))
            ymaxs.append(eval(o[4][3].text))
    return xmins, ymins, xmaxs, ymaxs


def bb_intersection_over_union(boxA, boxB):
    """Compute the intersection over union by taking the intersection area and dividing
    it by the sum of prediction + ground-truth areas - the interesection area"""
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    print(interArea)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    print(boxAArea)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    print(boxBArea)

    return interArea / float(boxAArea + boxBArea - interArea)


def find_matching_path(annotations, images):
    """Finds the corresponding image or annotation path (i.e. given on path searches for the corresponding path)"""
    if type(annotations) == str:
        anchor, search = annotations, images
    elif type(images) == str:
        anchor, search = images, annotations
    else:
        print("Somethings wrong")
    return [p for p in search if anchor.split("/")[-1].split(".")[0] in p][
        0
    ]  # only need one

#### Set paths

In [102]:
DOWNLOAD_DIR = os.path.join("/Users", "apowell", "Downloads")
assert os.path.isdir(DOWNLOAD_DIR)

DATA_DIR = os.path.join("..", "data", "Herring")

TRAINING_DIR = os.path.join(DATA_DIR, "Training", "IRWA 2017 (Large+annotations)")
TESTING_DIR = os.path.join(DATA_DIR, "Testing")

os.path.isdir(DATA_DIR)  # assertion that we are pointed to correct folder
WEIGHTS_DIR = os.path.join("..", "weights_and_config")
HERRING_WEIGHTS = os.path.join(WEIGHTS_DIR, "Herring")


folder = "Batch 1"
# Load images and annotations
# images_path1 = os.path.join(TRAINING_DIR, folder, "*.JPG")
# images_path2 = os.path.join(TRAINING_DIR, folder, "*.jpg")
images_path1 = os.path.join(DOWNLOAD_DIR, "HerringTestSet", "Images", "*.jpg")
images = glob.glob(images_path1)  # + glob.glob(images_path2)

# annotation_path = os.path.join(TRAINING_DIR, f"{folder} Annotations", "*.xml")
annotation_path = os.path.join(DOWNLOAD_DIR, "HerringTestSet", "Xml", "*.xml")
annotations = glob.glob(annotation_path)
print("{} images. Annotations: {}".format(len(images), len(annotations)))

94 images. Annotations: 94


In [113]:
# Example: Finding matching paths
test = images[0]
print("test:", test)
test_match = find_matching_path(images=test, annotations=annotations)
print("test match:", test_match)



test /Users/apowell/Downloads/HerringTestSet/Images/1_2016-04-21_21-50-1808716.jpg
test match: /Users/apowell/Downloads/HerringTestSet/Xml/1_2016-04-21_21-50-1808716.xml


## Scoring annotations against inference
**Requirements:** 2 folders: 1 with annotations (e.g. XML), 1 with images
* **Steps:**
  * 1) Run loop on images extracting predicted boxes from each image
  * 2) Scan filenames until box is properly extracted
  * 3) **Scoring Method:**
    * For each annotation find closest/any overlapping boxes (scoring the greater of number of either the annotated or predicted set distinctly)
    * Return average IOU for image
    * Return total number of correctly detected fish

In [67]:
def display_image_comparison(img_path=None, annotated_box=None, predicted_box=None):
    import matplotlib.pyplot as plt
    import cv2

    """Display an image with optional annotated class and predicted class within notebook for comparison.
    image_path: image path to location in memory
    annotated_box: corners in format xmin,xmax,ymin,ymax
    """

    label = "herring"

    img_name = img_path.split("/")[-1].split(".")[0]
    frame = cv2.imread(img_path)

    # Annotated
    cv2.rectangle(
        frame,
        (annotated_box[0], annotated_box[1]),
        (annotated_box[2], annotated_box[3]),
        (178, 255, 255),
        2,
    )

    # Annotated
    # Predicted
    if predicted_box is not None:
        cv2.rectangle(
            frame,
            (predicted_box[0], predicted_box[1]),
            (predicted_box[2], predicted_box[3]),
            (144, 238, 144),
            3,
        )
    cv2.putText(
        frame,
        "Annotated " + label,
        (annotated_box[0], annotated_box[1]),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.55,
        (178, 255, 255),
        1,
    )
    cv2.putText(
        frame,
        "Predicted " + label,
        (annotated_box[1], annotated_box[2]),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.55,
        (144, 238, 144),
        1,
    )
    plt.imshow(frame)
    plt.title(img_name)
    plt.show()


## Example files
# 1 count: ../data/Herring/Training/IRWA 2017 (Large+annotations)/Batch 1/2_2017-04-14_11-42-13_large.jpg
# herring_image = "../data/Herring/Training/IRWA 2017 (Large+annotations)/Batch 1/2_2017-04-14_11-42-13_large.jpg"
# boxes = [[56, 134, 97, 48]]

# 1 count: ../data/Herring/Training/IRWA 2017 (Large+annotations)/Batch 1/2_2017-04-14_10-51-06_large.jpg
herring_image = "../data/Herring/Training/IRWA 2017 (Large+annotations)/Batch 1/2_2017-04-14_10-51-06_large.jpg"
boxes = [[6, 110, 107, 56]]

print(herring_image, boxes)
# Get index of annotation path files that contains the path to annotated file matching the non-annoated file
idx = [
    i
    for i in range(len(annotations))
    if herring_image.split("/")[-1].split(".")[0] in annotations[i]
][
    0
]  # Get the first element (multiple matches do not matter)


xmins, xmaxs, ymins, ymaxs = get_dimensions_of_annotations(fpath=annotations[idx])

# Predicted
left, top, width, height = boxes[0][0], boxes[0][1], boxes[0][2], boxes[0][3]
xmin, ymin, xmax, ymax = left, top, left + width, top + height
predicted_box = xmin, ymin, xmax, ymax
# Annotated
annotated_box = xmins[0], xmaxs[0], ymins[0], ymaxs[0]
print("IOU Score:", bb_intersection_over_union(boxA=predicted_box, boxB=annotated_box))

display_image_comparison(
    img_path=herring_image, annotated_box=annotated_box, predicted_box=predicted_box
)  # predicted_box

../data/Herring/Training/IRWA 2017 (Large+annotations)/Batch 1/2_2017-04-14_10-51-06_large.jpg [[6, 110, 107, 56]]


IndexError: list index out of range

## Evaluation

In [103]:
## Initialize the parameters
# Confidence threshold
conf_threshold = 0.5
# Non-maximum suppression threshold (maximum bounding box)
nms_threshold = 0.05
input_width = 416  # Width of network's input image
input_height = 416  # Height of network's input image

# Load class name
classes = "Herring"
# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = os.path.join(HERRING_WEIGHTS, "herring.cfg")
modelWeights = os.path.join(HERRING_WEIGHTS, "herring_final.weights")
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)


iou_scores = []
annotated_counts = []
predicted_counts = []
# Loop through annotated files
for a in tqdm(annotations):
    name = a.split("/")[-1].split(".")[0]

    # Loop through image files (non-annotated) to make a prediction on
    for herring_image in images:

        if name not in herring_image:
            continue

        frame = cv2.imread(herring_image)

        # # Get frame from the video
        # hasFrame, frame = cap.read()

        input_width, input_height, _ = frame.shape
        input_width = input_height = min(input_width, input_height)
        input_width = 416  # Width of network's input image
        input_height = 416  # Height of network's input image

        print(input_height, input_width)
        # Create a 4D blob from a frame.
        blob = cv2.dnn.blobFromImage(
            frame, 1 / 255, (input_width, input_height), [0, 0, 0], 1, crop=False
        )

        # Sets the input to the network
        net.setInput(blob)

        # Runs the forward pass to get output of the output layers
        outs = net.forward(inference_utils.get_outputs_names(net=net))
        # break
        # Remove the bounding boxes with low confidence
        counts, boxes = inference_utils.postprocess(
            frame=frame,
            outs=outs,
            tracker=None,
            conf_threshold=conf_threshold,
            nms_threshold=nms_threshold,
            classes=classes,
        )

        print(boxes)
        # cv2.imshow(herring_image, frame)
        # cv2.waitKey(0)
        print("Counts:", counts)

        # Get score
        xmins, xmaxs, ymins, ymaxs = get_dimensions_of_annotations(fpath=a)
        # Predicted
        if len(boxes) == 0:  # If nothing was predicted
            boxes = [[0, 0, 0, 0]]
        left, top, width, height = boxes[0][0], boxes[0][1], boxes[0][2], boxes[0][3]
        xmin, ymin, xmax, ymax = left, top, left + width, top + height
        predicted_box = xmin, ymin, xmax, ymax
        # Annotated
        if len(xmins) == 0:  # If nothing was annotated
            annotated_box = [0, 0, 0, 0]
            annotated_count = 0
        else:
            annotated_box = xmins[0], xmaxs[0], ymins[0], ymaxs[0]
            annotated_count = len(xmins)

        annotated_counts.append(annotated_count)
        predicted_counts.append(counts)
        iou_score = bb_intersection_over_union(boxA=predicted_box, boxB=annotated_box)
        iou_scores.append(iou_score)
        print("IOU Score:", iou_score)

        print()
    cv2.destroyAllWindows()

  0%|          | 0/94 [00:00<?, ?it/s]

416 416


  2%|▏         | 2/94 [00:00<00:23,  3.98it/s]

[0.37076646 0.62617767 0.5142306  0.30055088 0.94516844 0.94516844]
BOXES [[72, 228, 329, 144]] [0] [0]
counts 1
[[72, 228, 329, 144]]
Counts: 1
45175
47850
45731
IOU Score: 0.9332520761889022

416 416
[0.7232271  0.66355705 0.543609   0.26670778 0.909749   0.909749  ]
BOXES [[288, 254, 347, 128]] [0] [0]
counts 1
[[288, 254, 347, 128]]
Counts: 1
36176
44892
36624
IOU Score: 0.7978826643140715

416 416


  3%|▎         | 3/94 [00:00<00:23,  3.90it/s]

...

[0.6635752  0.70139575 0.5403904  0.24297749 0.9289744  0.9289744 ]
BOXES [[251, 278, 345, 116]] [0] [0]
counts 1
[[251, 278, 345, 116]]
Counts: 1
36270
40482
38750
IOU Score: 0.8442344397374424

416 416
[0.48869583 0.58961284 0.4343481  0.2481359  0.92749214 0.92749214]
[0.10642012 0.6383523  0.20666902 0.2030523  0.9095578  0.9095578 ]
BOXES [[173, 223, 277, 119], [2, 257, 132, 97]] [0 1] [0, 0]
counts 2
[[173, 223, 277, 119], [2, 257, 132, 97]]
Counts: 2
31860
33360
33210
IOU Score: 0.9178910976663786

416 416


 99%|█████████▉| 93/94 [00:19<00:00,  5.63it/s]

[0.9112899  0.50406724 0.16773257 0.15382396 0.88390905 0.88390905]
BOXES [[529, 204, 107, 73]] [0] [0]
counts 1
[[529, 204, 107, 73]]
Counts: 1
6732
7992
7004
IOU Score: 0.8146176185866408

416 416
[0.32009712 0.59073603 0.39301378 0.16648228 0.9256019  0.9256019 ]
BOXES [[78, 243, 251, 79]] [0] [0]
counts 1
[[78, 243, 251, 79]]
Counts: 1
19600
20160
21070
IOU Score: 0.9061488673139159

416 416


100%|██████████| 94/94 [00:19<00:00,  4.84it/s]

[0.3289839  0.71345925 0.522956   0.280719   0.9146465  0.9146465 ]
BOXES [[43, 275, 334, 134]] [0] [0]
counts 1
[[43, 275, 334, 134]]
Counts: 1
36624
45225
36624
IOU Score: 0.8098175787728027



In [104]:
print("average IOU:", np.mean(iou_scores))
print("Total annotated counts:", sum(annotated_counts))
print("Total predicted counts:", sum(predicted_counts))
print(
    f"Ratio of predicted to expected: { sum(predicted_counts) / sum(annotated_counts):.3f}"
)

average IOU: 0.805932661840877
Total annotated counts: 112
Total predicted counts: 105
Ratio of predicted to expected: 0.938
